In [ ]:
RESULTS_PATH = '../results/'

In [ ]:
import numpy as np
import inputs
import classification
import itertools
from sklearn.preprocessing import StandardScaler, MinMaxScaler

Select the parameters and models for training, validation and testing

In [ ]:
task_list = [inputs.binary, inputs.six_transient, inputs.seven_transient, inputs.seven_class, inputs.eight_class]
min_obs_list = [5,10]
num_features_list = [31, 27, 21]
oversample_list = [True, False]
model_list = [classification.svc, classification.rf, classification.mlp]
scaler_list = [StandardScaler, MinMaxScaler]

For every combination of parameters selected, perform pre-processing and classification

In [ ]:
for combination in itertools.product(task_list, min_obs_list, num_features_list, oversample_list, model_list, scaler_list):
    task, min_obs, num_features, oversample, model, scaler = combination
    print('STARTING TASK: ', task.__name__, min_obs, num_features, oversample, model.__name__, scaler.__name__)
    # Obtain inputs
    X_train, X_test, y_train, y_test = task(min_obs, num_features, oversample=oversample)
    # Scale inputs
    scaler = scaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    # Perform Classification
    model(X_train, y_train, X_test, y_test, min_obs, num_features, oversample, task, scaler)
    print('Finished Task\n')